Ogólny przegląd danych ściągniętych z StatsBomb przechowywanych w bazie MongoDB.

In [1]:
import pandas as pd
from statsbombpy import sb
from pymongo import MongoClient

In [2]:
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "football_data"
MATCHES_COLLECTION = "matches"
EVENTS_COLLECTION = "events"

client = MongoClient(MONGO_URI)
db = client[DB_NAME]

In [12]:
#gole strzelone przez danego zawodnika
player_name = "Lionel Andrés Messi Cuccittini"
query = {
    "type": "Shot",
    "shot_outcome": "Goal",
    "player": player_name
}
projection = {
    "_id": 0,
    "match_id": 1,
    "minute": 1,
    "team": 1,
}
messi_goals_df = pd.DataFrame(list(db[EVENTS_COLLECTION].find(query, projection)))
print(messi_goals_df.head())

   match_id  minute       team
0    266498      39  Barcelona
1    266310      72  Barcelona
2    267576      29  Barcelona
3    266653      22  Barcelona
4    266653      52  Barcelona


In [13]:
#dodanie kolumny z informacją przeciwko komu strzelił
matches_df = pd.DataFrame(list(db[MATCHES_COLLECTION].find({}, {"_id": 0, "match_id": 1, "home_team": 1, "away_team": 1})))
messi_goals_df = messi_goals_df.merge(matches_df, on="match_id", how="left")
messi_goals_df["opponent"] = messi_goals_df.apply(
    lambda row: row["away_team"] if row["team"] == row["home_team"] else row["home_team"], axis=1
)

messi_goals_df = messi_goals_df.drop(columns=["home_team", "away_team"])

print(messi_goals_df.head())

   match_id  minute       team                opponent
0    266498      39  Barcelona                  Getafe
1    266310      72  Barcelona  RC Deportivo La Coruña
2    267576      29  Barcelona         Atlético Madrid
3    266653      22  Barcelona          Rayo Vallecano
4    266653      52  Barcelona          Rayo Vallecano


In [ ]:
#gole z danego meczu Real Madrid vs Rayo Vallecano
match_query = {"home_team": "Real Madrid", "away_team": "Rayo Vallecano"}
match = db[MATCHES_COLLECTION].find_one(match_query, {"_id": 0, "match_id": 1})

match_id = match["match_id"]

goal_query = {
    "match_id": match_id,
    "type": "Shot",
    "shot_outcome": "Goal"
}

projection = {
    "_id": 0, 
    "match_id": 1,
    "minute": 1,
    "player": 1,
    "team": 1,

}
goals_df = pd.DataFrame(list(db[EVENTS_COLLECTION].find(goal_query, projection)))

goals_df

,match_id,minute,player,team
0,3825700,2,Danilo Luiz da Silva,Real Madrid
1,3825700,9,Antonio Amaya Carazo,Rayo Vallecano
2,3825700,11,Jozabed Sánchez Ruiz,Rayo Vallecano
3,3825700,24,Gareth Frank Bale,Real Madrid
4,3825700,29,Cristiano Ronaldo dos Santos Aveiro,Real Madrid
5,3825700,40,Gareth Frank Bale,Real Madrid
6,3825700,47,Karim Benzema,Real Madrid
7,3825700,52,Cristiano Ronaldo dos Santos Aveiro,Real Madrid
8,3825700,60,Gareth Frank Bale,Real Madrid
9,3825700,69,Gareth Frank Bale,Real Madrid
